# HRRR 2m Temperature Analysis

This notebook demonstrates how to load, process, and visualize 2-meter temperature data from HRRR using Xarray, Cartopy, and Matplotlib.

## Setup

We'll start by importing the necessary libraries.

In [ ]:

import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import metpy.calc as mpcalc
from metpy.units import units
from pelicanfs.core import OSDFFileSystem

## Define File Paths

Define the date, hour, variable, and level for the HRRR data.

In [ ]:
# Set the date, hour, variable, and level for the HRRR data
date = '20211016'
hour = '21'
var = 'TMP'
level = '2m_above_ground'

# Construct file paths for the Zarr datasets
namespace_file1 = f'/chtc/PUBLIC/eturetsky/hrrrzarr/sfc/{date}/{date}_{hour}z_anl.zarr/{level}/{var}/{level}/'
namespace_file2 = f'/chtc/PUBLIC/eturetsky/hrrrzarr/sfc/{date}/{date}_{hour}z_anl.zarr/{level}/{var}/'


## Open Zarr Datasets

Use OSDFFileSystem to open the Zarr datasets and read them with Xarray.

In [ ]:
# Initialize the OSDFFileSystem
xfs = OSDFFileSystem()

# Get mappers for the Zarr datasets
file1 = xfs.get_mapper(namespace_file1)
file2 = xfs.get_mapper(namespace_file2)

# Open the datasets
ds = xr.open_mfdataset([file1, file2], engine='zarr')

# Display the dataset
ds

## Define Projection and Coordinate System

Set up the projection and coordinate system for plotting.

In [ ]:
# Define coordinates for projection
lon1 = -97.5
lat1 = 38.5
slat = 38.5

# Define the Lambert Conformal projection
projData = ccrs.LambertConformal(
    central_longitude=lon1,
    central_latitude=lat1,
    standard_parallels=[slat, slat],
    globe=ccrs.Globe(
        semimajor_axis=6371229,
        semiminor_axis=6371229
    )
)

# Display dataset coordinates
ds.coords

## Extract and Convert Data

Extract temperature data and convert its units

In [ ]:
# Extract temperature data
airTemp = ds.TMP

# Display the temperature data
airTemp

# Convert temperature units to Celsius
airTemp = airTemp.metpy.convert_units('degC')

# Display the converted temperature data
airTemp


## Plot Temperature Data

Create a plot of the temperature data.

In [ ]:
# Extract projection coordinates
x = airTemp.projection_x_coordinate
y = airTemp.projection_y_coordinate

# Plot temperature data
airTemp.plot(figsize=(11, 8.5))

# Compute minimum and maximum temperatures
minTemp = airTemp.min().compute()
maxTemp = airTemp.max().compute()

# Display minimum and maximum temperature values
minTemp.values, maxTemp.values


## Define Contour Levels and Plot

Set up contour levels and plot the temperature data.

In [ ]:
# Define contour levels
fint = np.arange(np.floor(minTemp.values), np.ceil(maxTemp.values) + 2, 2)

# Define plot bounds and resolution
latN = 50.4
latS = 24.25
lonW = -123.8
lonE = -71.2
res = '50m'

# Create a figure and axis with projection
fig = plt.figure(figsize=(18, 12))
ax = plt.subplot(1, 1, 1, projection=projData)
ax.set_extent([lonW, lonE, latS, latN], crs=ccrs.PlateCarree())
ax.add_feature(cfeature.COASTLINE.with_scale(res))
ax.add_feature(cfeature.STATES.with_scale(res))

# Add the title
tl1 = 'HRRR 2m temperature ($^\circ$C)'
tl2 = f'Analysis valid at: {hour}00 UTC {date}'
plt.title(f'{tl1}\n{tl2}', fontsize=16)

# Contour fill
CF = ax.contourf(x, y, airTemp, levels=fint, cmap=plt.get_cmap('coolwarm'))

# Make a colorbar for the ContourSet returned by the contourf call
cbar = fig.colorbar(CF, shrink=0.5)
cbar.set_label(r'2m Temperature ($^\circ$C)', size='large')

# Show the plot
plt.show()